# Previsão de Séries Temporais com Programação Genética
Este exercício tem o objetivo de utilizar Programaçã Genética para gerar um modelo de previsão de séries temporais. Os dados utilizados são referente ao número mensal de passageirosde linhas aéreas internacionais (em milhares/mês), desde janeir/1949 a dezembro/1960, conforme figura abaixo.
![grafico](serie-grafico.png)

O primeiro passo foi criar o dataset com os dados da planilha fornecida. Separou-se em dois datasets: treinamento e teste.

Para resolução dessa atividdade será o DEAP(Distributed Evolutionary Algorithms in Python)

Primeiramente, transformou-se a planilha fornecida em dois conjuntos de dadoscarregados abaxi

In [10]:
#    This file is part of EAP.
#
#    EAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    EAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with EAP. If not, see <http://www.gnu.org/licenses/>.

import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof


Exception: Ephemerals with different functions should be named differently, even between psets.

In [9]:
if __name__ == "__main__":
    pop, log, hof = main()

   	      	                fitness                	              size             
   	      	---------------------------------------	-------------------------------
gen	nevals	avg    	max  	min     	std    	avg    	max	min	std    
0  	300   	1.78879	30.34	0.450825	2.67896	3.54667	7  	2  	1.49482
1  	166   	1.43254	44.4437	0.183711	3.05668	3.60667	12 	1  	1.77725
2  	166   	2.16879	315.736	0.165572	18.1873	3.55   	9  	1  	1.62506
3  	163   	0.98255	2.9829 	0.165572	0.712666	3.42667	9  	1  	1.45073
4  	153   	0.836017	14.538 	0.165572	0.979399	3.77   	11 	1  	1.64025
5  	158   	0.944635	18.9739	0.165572	1.61614 	3.77667	10 	1  	1.62894
6  	169   	0.885819	14.2181	0.165572	1.00296 	4      	10 	1  	1.87617
7  	167   	0.731332	3.35292	0.165572	0.56016 	4.35   	10 	1  	1.92722
8  	187   	0.785916	19.1852	0.13008 	1.2426  	5.13667	14 	1  	2.21465
9  	176   	0.672788	14.2339	0.123719	1.00764 	5.71667	14 	1  	2.48389
10 	176   	0.786975	32.4952	0.123719	2.56679 	6.27333	15 	1  	2.33922
11 	167

In [25]:
for p in pop:
    stringify(p)

NameError: name 'stringify' is not defined

In [23]:
func

<function <lambda>>